<a href="https://www.kaggle.com/code/marcusngjh/store-sales-eda?scriptVersionId=101248895" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import Relevant Libraries 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_theme(style='ticks')
%matplotlib inline 

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
hols = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
trans = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')

Interpolate oil prices before merging, to avoid large missing values

In [3]:
oil['date'] = pd.to_datetime(oil['date'])

x = oil['date']
y = oil['dcoilwtico']

fig = px.line(oil, x, y, title='Oil Prices')
fig.show()

oil['dcoilwtico'] = oil['dcoilwtico'].interpolate()

x = oil['date']
y = oil['dcoilwtico']

fig = px.line(oil, x, y, title='Oil Prices with Interpolation')
fig.show()

### Merging all DataFrames 

In [4]:
df = train.merge(hols, on = 'date', how='left')
df = df.merge(stores, on = 'store_nbr', how='left')
df = df.merge(trans, on = ['date', 'store_nbr'], how='left')
df = pd.concat([df, test], ignore_index=True) # stack below train df, according to date 
df = df.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})


df['date'] = pd.to_datetime(df['date'])
df = df.merge(oil, on = 'date', how='left') # oil is already datetime formatted, hence we merge after to avoid errors
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week
df['quarter'] = df['date'].dt.quarter
df['day_of_week'] = df['date'].dt.day_name()
df.head()

,id,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,...,state,store_type,cluster,transactions,dcoilwtico,year,month,week,quarter,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,Pichincha,D,13.0,NaN,NaN,2013,1,1,1,Tuesday
1,1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,Pichincha,D,13.0,NaN,NaN,2013,1,1,1,Tuesday
2,2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,Pichincha,D,13.0,NaN,NaN,2013,1,1,1,Tuesday
3,3,2013-01-01,1,BEVERAGES,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,Pichincha,D,13.0,NaN,NaN,2013,1,1,1,Tuesday
4,4,2013-01-01,1,BOOKS,0.0,0,Holiday,National,Ecuador,Primer dia del ano,...,Pichincha,D,13.0,NaN,NaN,2013,1,1,1,Tuesday


In [5]:
df = df.set_index('id')
df.head(1)

,date,store_nbr,family,sales,onpromotion,holiday_type,locale,locale_name,description,transferred,...,state,store_type,cluster,transactions,dcoilwtico,year,month,week,quarter,day_of_week
id,,,,,,,,,,,,,,,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Ecuador,Primer dia del ano,False,...,Pichincha,D,13.0,NaN,NaN,2013,1,1,1,Tuesday


In [6]:
df['dcoilwtico'] = df['dcoilwtico'].interpolate()

x = oil['date']
y = oil['dcoilwtico']

fig = px.line(oil, x, y, title='Merged Oil Prices with Time Interpolation')
fig.show()

To avoid false positives, only stick to national holidays. Clearer representation of store sales per holiday. <br>
Drop columns with additional information. 

In [7]:
df['locale'].unique()

array(['National', nan, 'Local', 'Regional'], dtype=object)

In [8]:
df = df.loc[df['locale']=='National']
df = df.drop(['locale_name', 'description', 'transferred', 'state'], axis=1)
df.head(3)

,date,store_nbr,family,sales,onpromotion,holiday_type,locale,city,store_type,cluster,transactions,dcoilwtico,year,month,week,quarter,day_of_week
id,,,,,,,,,,,,,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0,Holiday,National,Quito,D,13.0,NaN,NaN,2013,1,1,1,Tuesday
1,2013-01-01,1,BABY CARE,0.0,0,Holiday,National,Quito,D,13.0,NaN,NaN,2013,1,1,1,Tuesday
2,2013-01-01,1,BEAUTY,0.0,0,Holiday,National,Quito,D,13.0,NaN,NaN,2013,1,1,1,Tuesday


In [9]:
df.isnull().sum()

date                0
store_nbr           0
family              0
sales               0
onpromotion         0
holiday_type        0
locale              0
city                0
store_type          0
cluster             0
transactions    25773
dcoilwtico       1782
year                0
month               0
week                0
quarter             0
day_of_week         0
dtype: int64

No oil price information at the start of year - drop NaN values. 
Same goes for transactions. 

In [10]:
df['transactions'].fillna(0, inplace=True)
df['dcoilwtico'].fillna(0, inplace=True)
df[['transactions', 'dcoilwtico']].isnull().sum()

transactions    0
dcoilwtico      0
dtype: int64

### Convert Weekdays into Numerical Data

In [11]:
day_of_week = {'Monday': 1, 'Tuesday':2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
df['day_of_week'] = df['day_of_week'].map(day_of_week)
df['day_of_week'].unique()

array([2, 6, 1, 3, 7, 5, 4])

In [12]:
fig = make_subplots(rows=5, cols=1, 
                    subplot_titles=('Yearly', 'Monthly', 'Weekly', 'Quarterly', 
                                    'Day of Week'))

def avr_sales(col):

    avr = df.groupby(col).agg({'sales': 'mean'}).reset_index()
    
    trace = go.Scatter(x = avr[col], y = avr['sales'])

    return trace

trace_1 = avr_sales('year')
trace_2 = avr_sales('month')
trace_3 = avr_sales('week')
trace_4 = avr_sales('quarter')
trace_5 = avr_sales('day_of_week')

fig.add_trace(trace_1, row=1, col=1)
fig.add_trace(trace_2, row=2, col=1)
fig.add_trace(trace_3, row=3, col=1)
fig.add_trace(trace_4, row=4, col=1)
fig.add_trace(trace_5, row=5, col=1)


fig.update_layout(height=800, title_text = 'Average Sales Per Period', showlegend=False, 
                xaxis = dict(tickmode='linear', tick0=2013, dtick=1), 
                xaxis4 = dict(tickmode='linear', tick0=1, dtick=1)
)
fig.show()

In [13]:
avr = df.groupby(['month', 'store_type']).agg({'sales': 'mean'}).reset_index()

fig = px.line(avr, x = 'month', y = 'sales', color='store_type', 
                title='Average Sales by Store Type')

fig.update_xaxes(tickmode='linear')
fig.show()

### Analyse Average Sales by Store Numbers, split between work and holidays

In [14]:
df['holiday_type'].unique()

array(['Holiday', 'Work Day', 'Additional', 'Event', 'Transfer', 'Bridge'],
      dtype=object)

In [15]:
def check_hol(row):
    if row != 'Work Day':
        value = 'Holiday'
    else: 
        value = row
    
    return value 

df['Type of Day'] = df['holiday_type'].apply(check_hol)
df['Type of Day'].unique()

array(['Holiday', 'Work Day'], dtype=object)

In [16]:
store_sales = df.groupby(['Type of Day', 'store_nbr']).agg({'sales': 'mean'}).reset_index()

fig = px.bar(store_sales, x = 'store_nbr', y = 'sales', 
            color = 'Type of Day', title='Average Sales by Store Number')

fig.show()

fig = px.pie(store_sales, values = 'sales', names = 'Type of Day', title = 'Average Sales - Type of Day')

            
fig.show()

In [17]:
# Extract yearly data from year 2017
sales_2017 = df.loc[df['year']==2017]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

onpromotion = sales_2017['onpromotion']
sales = sales_2017['sales']
transactions = sales_2017['transactions']

# Add traces
fig.add_trace(
    go.Scatter(x=onpromotion, y=sales, mode='markers', name="sales based on promotion"),
                secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=onpromotion, y=transactions, mode='markers', name="transactions based on promotion",
                marker=dict(opacity=0.5)), 
                secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Store promotion vs Sales/Transactions 2017"
)

# Set x-axis title
fig.update_xaxes(title_text="Number of items promoted at any given date")

# Set y-axes titles
fig.update_yaxes(title_text="Sales", secondary_y=False)
fig.update_yaxes(title_text="Transactions", secondary_y=True)

fig.show()

### Plot of correlation matrix using only numerical data. 

In [18]:
corr = df[['sales', 'onpromotion', 'transactions', 'dcoilwtico']]
fig = px.imshow(corr.corr())
fig.show()

### Oil Prices has little to no correlation to Store Sales

In [19]:
agg = df.groupby('date').agg({"sales" : "mean"}).reset_index()
fig = px.line(agg, x='date', y="sales")
fig.update_layout(title = "Average Sales by Date")
fig.show()

agg2 = df.groupby('date').agg({"dcoilwtico" : "mean"}).reset_index()
fig = px.line(agg2, x='date', y="dcoilwtico")
fig.update_layout(title = "Oil Prices by Date")
fig.show()